# PACT-Based Multi-Agent Paper Critique System

This notebook implements a multi-agent system for critiquing student papers using the PACT (PennCLO Academic Critique Taxonomy) framework.

## Architecture Overview

The system consists of:
1. **5 Specialized PACT Agents** - Each focused on one dimension of the PACT taxonomy
2. **Supervisor Agent** - Coordinates the specialized agents and synthesizes their feedback
3. **Input Processing** - Handles student paper submission and parsing
4. **Output Generation** - Creates comprehensive, structured feedback reports

### PACT Dimensions:
1. **Research Foundations** - Problem definition, frameworks, literature
2. **Methodological Rigor** - Methods, data, analysis, limitations
3. **Structure & Coherence** - Organization, flow, transitions
4. **Academic Precision** - Terms, citations, grammar, formatting
5. **Critical Sophistication** - Reflexivity, originality, theoretical depth

In [ ]:
# Load environment variables and set up auto-reload
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

## Load and Parse PACT Taxonomy

In [ ]:
%%writefile ../src/pact/pact_taxonomy.py

"""
PACT Taxonomy Loader and Parser

This module loads and structures the PACT taxonomy for use by the critique agents.
"""

import json
import zipfile
import xml.etree.ElementTree as ET
from typing import Dict, Any, List
from pathlib import Path

def load_pact_taxonomy(file_path: str = "../PACT_JSON.docx") -> Dict[str, Any]:
    """
    Load the PACT taxonomy from the docx file.
    
    Returns a structured dictionary with the taxonomy dimensions.
    """
    # Check if we already have a parsed JSON version
    json_path = Path("../pact_taxonomy.json")
    if json_path.exists():
        with open(json_path, 'r') as f:
            return json.load(f)
    
    # Otherwise, parse from docx
    with zipfile.ZipFile(file_path, 'r') as docx:
        xml_content = docx.read('word/document.xml')
        tree = ET.fromstring(xml_content)
        
        namespace = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
        paragraphs = []
        for para in tree.iter(namespace + 'p'):
            texts = [node.text for node in para.iter(namespace + 't') if node.text]
            if texts:
                paragraphs.append(''.join(texts))
        
        # Parse JSON from text
        full_text = '\n'.join(paragraphs)
        json_start = full_text.find('{')
        if json_start != -1:
            json_text = full_text[json_start:]
            # Find matching closing brace
            brace_count = 0
            end_pos = 0
            for i, char in enumerate(json_text):
                if char == '{':
                    brace_count += 1
                elif char == '}':
                    brace_count -= 1
                    if brace_count == 0:
                        end_pos = i + 1
                        break
            
            json_text = json_text[:end_pos]
            pact_data = json.loads(json_text)
            
            # Save for future use
            with open(json_path, 'w') as f:
                json.dump(pact_data, f, indent=2)
            
            return pact_data
    
    raise ValueError("Could not parse PACT taxonomy from file")

def get_dimension_details(pact_data: Dict[str, Any], dimension_id: str) -> Dict[str, Any]:
    """
    Extract details for a specific PACT dimension.
    
    Args:
        pact_data: The full PACT taxonomy data
        dimension_id: The dimension ID (e.g., "1.0.0")
    
    Returns:
        Dictionary with dimension details including subsections
    """
    dimensions = pact_data.get('dimensions', {})
    return dimensions.get(dimension_id, {})

def get_all_dimensions(pact_data: Dict[str, Any]) -> List[tuple]:
    """
    Get all main dimensions from the PACT taxonomy.
    
    Returns:
        List of (dimension_id, dimension_name, dimension_data) tuples
    """
    dimensions = pact_data.get('dimensions', {})
    main_dimensions = []
    
    for dim_id in ['1.0.0', '2.0.0', '3.0.0', '4.0.0', '5.0.0']:
        if dim_id in dimensions:
            dim_data = dimensions[dim_id]
            main_dimensions.append((dim_id, dim_data.get('name'), dim_data))
    
    return main_dimensions

## Define State and Schemas for Critique System

In [ ]:
%%writefile ../src/pact/state_pact_critique.py

"""
State Definitions and Schemas for PACT Critique System

This module defines the state objects and structured schemas used for
the multi-agent paper critique workflow.
"""

import operator
from typing import Optional, List, Dict, Any
from typing_extensions import Annotated

from pydantic import BaseModel, Field
from langgraph.graph import MessagesState

# ===== STATE DEFINITIONS =====

class PaperCritiqueState(MessagesState):
    """
    Main state for the PACT critique system.
    
    Tracks the paper being critiqued, individual agent feedback,
    and the final synthesized critique.
    """
    # The student paper to critique
    paper_content: str
    
    # Paper metadata
    paper_title: Optional[str] = None
    paper_type: Optional[str] = None  # thesis, dissertation, article, etc.
    
    # Individual dimension critiques from specialized agents
    dimension_critiques: Annotated[Dict[str, Any], operator.add] = {}
    
    # Supervisor's analysis plan
    critique_plan: Optional[str] = None
    
    # Final synthesized critique
    final_critique: Optional[str] = None
    
    # Overall paper score (0-100)
    overall_score: Optional[float] = None
    
    # Priority areas for improvement
    priority_improvements: List[str] = []

# ===== STRUCTURED OUTPUT SCHEMAS =====

class DimensionCritique(BaseModel):
    """
    Schema for individual dimension critiques from specialized agents.
    """
    dimension_id: str = Field(
        description="The PACT dimension ID (e.g., '1.0.0')"
    )
    dimension_name: str = Field(
        description="The dimension name (e.g., 'Research Foundations')"
    )
    strengths: List[str] = Field(
        description="Specific strengths identified in this dimension",
        default_factory=list
    )
    weaknesses: List[str] = Field(
        description="Specific weaknesses or areas for improvement",
        default_factory=list
    )
    specific_issues: List[Dict[str, str]] = Field(
        description="Specific issues with location and severity",
        default_factory=list
    )
    recommendations: List[str] = Field(
        description="Actionable recommendations for improvement",
        default_factory=list
    )
    rubric_scores: Dict[str, int] = Field(
        description="Rubric scores for subsections (1-5 scale)",
        default_factory=dict
    )
    dimension_score: float = Field(
        description="Overall score for this dimension (0-100)",
        ge=0, le=100
    )
    severity: str = Field(
        description="Overall severity level: Critical, Major, Moderate, Minor",
        default="Moderate"
    )

class CritiquePlan(BaseModel):
    """
    Schema for the supervisor's critique plan.
    """
    paper_summary: str = Field(
        description="Brief summary of the paper's content and purpose"
    )
    initial_assessment: str = Field(
        description="Initial high-level assessment of paper quality"
    )
    dimensions_to_evaluate: List[str] = Field(
        description="List of PACT dimensions to evaluate",
        default_factory=lambda: ['1.0.0', '2.0.0', '3.0.0', '4.0.0', '5.0.0']
    )
    special_considerations: List[str] = Field(
        description="Any special considerations for this particular paper",
        default_factory=list
    )

class FinalCritique(BaseModel):
    """
    Schema for the final synthesized critique.
    """
    executive_summary: str = Field(
        description="Executive summary of the critique"
    )
    overall_assessment: str = Field(
        description="Overall assessment of the paper's quality"
    )
    dimension_summaries: Dict[str, str] = Field(
        description="Summary for each PACT dimension evaluated",
        default_factory=dict
    )
    key_strengths: List[str] = Field(
        description="Top 3-5 key strengths across all dimensions",
        default_factory=list
    )
    priority_improvements: List[str] = Field(
        description="Top 3-5 priority areas for improvement",
        default_factory=list
    )
    actionable_next_steps: List[str] = Field(
        description="Specific, actionable next steps for the author",
        default_factory=list
    )
    overall_score: float = Field(
        description="Overall paper score (0-100)",
        ge=0, le=100
    )
    recommendation: str = Field(
        description="Final recommendation: Accept, Revise, Major Revision, Reject"
    )

## Create Specialized PACT Dimension Agents

In [ ]:
%%writefile ../src/pact/pact_dimension_agents.py

"""
Specialized PACT Dimension Critique Agents

This module implements individual agents for each PACT dimension,
each specialized in evaluating specific aspects of academic writing.
"""

from typing import Dict, Any
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

from pact.state_pact_critique import PaperCritiqueState, DimensionCritique
from pact.pact_taxonomy import load_pact_taxonomy, get_dimension_details

# Initialize model for critique agents
critique_model = init_chat_model(model="openai:gpt-4.1", temperature=0.2)

def create_dimension_critique_prompt(paper_content: str, dimension_data: Dict[str, Any]) -> str:
    """
    Create a critique prompt for a specific PACT dimension.
    """
    return f"""
You are an expert academic reviewer specializing in evaluating the '{dimension_data.get('name')}' dimension of academic papers.

Your task is to critique the following paper according to the PACT taxonomy criteria for this dimension.

DIMENSION DETAILS:
Name: {dimension_data.get('name')}
Description: {dimension_data.get('description')}
Severity: {dimension_data.get('severity')}

EVALUATION CRITERIA:
{format_dimension_criteria(dimension_data)}

PAPER TO CRITIQUE:
---
{paper_content[:5000]}  # Truncate for context limits
---

Please provide a thorough critique focusing on:
1. Specific strengths in this dimension
2. Specific weaknesses or issues
3. Location of issues (paragraph/section references where possible)
4. Severity of each issue
5. Concrete, actionable recommendations for improvement
6. Rubric scores (1-5) for each subsection

Be constructive but honest. Focus on helping the author improve their work.
"""

def format_dimension_criteria(dimension_data: Dict[str, Any]) -> str:
    """
    Format the evaluation criteria for a dimension.
    """
    criteria = []
    sections = dimension_data.get('sections', {})
    
    for section_id, section_data in sections.items():
        criteria.append(f"\n{section_id}: {section_data.get('name')}")
        
        # Add subsections if available
        subsections = section_data.get('subsections', {})
        for subsection_id, subsection_data in subsections.items():
            criteria.append(f"  - {subsection_id}: {subsection_data.get('name')}")
            
            # Add detection patterns if available
            patterns = subsection_data.get('detection_patterns', [])
            if patterns:
                criteria.append(f"    Detection patterns: {', '.join(patterns[:3])}")
    
    return '\n'.join(criteria)

async def critique_dimension(state: PaperCritiqueState, dimension_id: str) -> Dict[str, Any]:
    """
    Critique a paper for a specific PACT dimension.
    
    Args:
        state: The current critique state
        dimension_id: The PACT dimension to evaluate (e.g., "1.0.0")
    
    Returns:
        Dictionary with the dimension critique
    """
    # Load PACT taxonomy
    pact_data = load_pact_taxonomy()
    dimension_data = get_dimension_details(pact_data, dimension_id)
    
    if not dimension_data:
        raise ValueError(f"Dimension {dimension_id} not found in PACT taxonomy")
    
    # Create critique prompt
    prompt = create_dimension_critique_prompt(state['paper_content'], dimension_data)
    
    # Get structured critique from model
    structured_model = critique_model.with_structured_output(DimensionCritique)
    critique = await structured_model.ainvoke([HumanMessage(content=prompt)])
    
    # Ensure dimension info is set
    critique.dimension_id = dimension_id
    critique.dimension_name = dimension_data.get('name', '')
    
    return critique.dict()

# Create specific agent functions for each dimension
async def critique_research_foundations(state: PaperCritiqueState) -> Dict[str, Any]:
    """Agent 1: Critique Research Foundations (1.0.0)"""
    critique = await critique_dimension(state, "1.0.0")
    return {"dimension_critiques": {"1.0.0": critique}}

async def critique_methodological_rigor(state: PaperCritiqueState) -> Dict[str, Any]:
    """Agent 2: Critique Methodological Rigor (2.0.0)"""
    critique = await critique_dimension(state, "2.0.0")
    return {"dimension_critiques": {"2.0.0": critique}}

async def critique_structure_coherence(state: PaperCritiqueState) -> Dict[str, Any]:
    """Agent 3: Critique Structure & Coherence (3.0.0)"""
    critique = await critique_dimension(state, "3.0.0")
    return {"dimension_critiques": {"3.0.0": critique}}

async def critique_academic_precision(state: PaperCritiqueState) -> Dict[str, Any]:
    """Agent 4: Critique Academic Precision (4.0.0)"""
    critique = await critique_dimension(state, "4.0.0")
    return {"dimension_critiques": {"4.0.0": critique}}

async def critique_critical_sophistication(state: PaperCritiqueState) -> Dict[str, Any]:
    """Agent 5: Critique Critical Sophistication (5.0.0)"""
    critique = await critique_dimension(state, "5.0.0")
    return {"dimension_critiques": {"5.0.0": critique}}

## Create the Supervisor Agent

In [ ]:
%%writefile ../src/pact/pact_supervisor.py

"""
PACT Critique Supervisor Agent

This module implements the supervisor agent that coordinates the specialized
PACT dimension agents and synthesizes their feedback into a cohesive critique.
"""

from typing import Dict, Any, Literal
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langgraph.types import Command

from pact.state_pact_critique import (
    PaperCritiqueState, CritiquePlan, FinalCritique
)

# Initialize supervisor model
supervisor_model = init_chat_model(model="openai:gpt-4.1", temperature=0.1)

def create_planning_prompt(paper_content: str) -> str:
    """
    Create a prompt for the supervisor to plan the critique.
    """
    return f"""
You are the lead reviewer coordinating a comprehensive academic paper critique using the PACT taxonomy.

Review the following paper and create a critique plan:

PAPER:
---
{paper_content[:3000]}  # Show first part for initial assessment
---

Create a plan that includes:
1. A brief summary of the paper's content and purpose
2. An initial assessment of overall quality and key areas of concern
3. Which PACT dimensions are most relevant to evaluate (all 5 by default)
4. Any special considerations for this particular paper

The PACT dimensions are:
1.0.0 - Research Foundations (problem, framework, literature)
2.0.0 - Methodological Rigor (methods, data, analysis)
3.0.0 - Structure & Coherence (organization, flow, transitions)
4.0.0 - Academic Precision (terms, citations, grammar)
5.0.0 - Critical Sophistication (reflexivity, originality, theory)
"""

def create_synthesis_prompt(state: PaperCritiqueState) -> str:
    """
    Create a prompt for synthesizing all dimension critiques.
    """
    # Format dimension critiques
    critiques_text = ""
    for dim_id, critique in state['dimension_critiques'].items():
        critiques_text += f"\n\n--- {critique['dimension_name']} ({dim_id}) ---\n"
        critiques_text += f"Score: {critique['dimension_score']}/100\n"
        critiques_text += f"Severity: {critique['severity']}\n"
        
        if critique['strengths']:
            critiques_text += f"Strengths:\n"
            for strength in critique['strengths']:
                critiques_text += f"  • {strength}\n"
        
        if critique['weaknesses']:
            critiques_text += f"Weaknesses:\n"
            for weakness in critique['weaknesses']:
                critiques_text += f"  • {weakness}\n"
        
        if critique['recommendations']:
            critiques_text += f"Recommendations:\n"
            for rec in critique['recommendations']:
                critiques_text += f"  • {rec}\n"
    
    return f"""
You are synthesizing feedback from multiple expert reviewers into a cohesive, actionable critique.

CRITIQUE PLAN:
{state.get('critique_plan', 'No plan available')}

INDIVIDUAL DIMENSION CRITIQUES:
{critiques_text}

Create a comprehensive final critique that:
1. Provides an executive summary of the paper's overall quality
2. Synthesizes feedback across all dimensions
3. Identifies the top 3-5 key strengths
4. Identifies the top 3-5 priority areas for improvement
5. Provides specific, actionable next steps
6. Calculates an overall score (weighted average of dimension scores)
7. Makes a final recommendation (Accept, Revise, Major Revision, Reject)

Be constructive and supportive while maintaining academic rigor.
Focus on helping the author improve their work.
"""

async def plan_critique(state: PaperCritiqueState) -> Command[Literal["evaluate_dimensions"]]:
    """
    Supervisor plans the critique approach.
    """
    # Create planning prompt
    prompt = create_planning_prompt(state['paper_content'])
    
    # Get structured plan from model
    structured_model = supervisor_model.with_structured_output(CritiquePlan)
    plan = await structured_model.ainvoke([HumanMessage(content=prompt)])
    
    # Format plan as string for state
    plan_text = f"""
Paper Summary: {plan.paper_summary}

Initial Assessment: {plan.initial_assessment}

Dimensions to Evaluate: {', '.join(plan.dimensions_to_evaluate)}

Special Considerations:
{"; ".join(plan.special_considerations) if plan.special_considerations else "None"}
"""
    
    return Command(
        goto="evaluate_dimensions",
        update={"critique_plan": plan_text}
    )

async def synthesize_critique(state: PaperCritiqueState) -> Dict[str, Any]:
    """
    Supervisor synthesizes all dimension critiques into final feedback.
    """
    # Create synthesis prompt
    prompt = create_synthesis_prompt(state)
    
    # Get structured final critique from model
    structured_model = supervisor_model.with_structured_output(FinalCritique)
    final_critique = await structured_model.ainvoke([HumanMessage(content=prompt)])
    
    # Format final critique as markdown
    critique_text = f"""
# Academic Paper Critique Report

## Executive Summary
{final_critique.executive_summary}

## Overall Assessment
{final_critique.overall_assessment}

**Overall Score:** {final_critique.overall_score}/100
**Recommendation:** {final_critique.recommendation}

## Dimension Evaluations
"""
    
    for dim_id, summary in final_critique.dimension_summaries.items():
        critique_text += f"\n### {dim_id}
{summary}\n"
    
    critique_text += f"""
## Key Strengths
"""
    for strength in final_critique.key_strengths:
        critique_text += f"- {strength}\n"
    
    critique_text += f"""
## Priority Areas for Improvement
"""
    for improvement in final_critique.priority_improvements:
        critique_text += f"- {improvement}\n"
    
    critique_text += f"""
## Actionable Next Steps
"""
    for i, step in enumerate(final_critique.actionable_next_steps, 1):
        critique_text += f"{i}. {step}\n"
    
    return {
        "final_critique": critique_text,
        "overall_score": final_critique.overall_score,
        "priority_improvements": final_critique.priority_improvements,
        "messages": [f"Critique complete. Overall score: {final_critique.overall_score}/100"]
    }

## Build the Complete PACT Critique Workflow

In [ ]:
%%writefile ../src/pact/pact_critique_agent.py

"""
PACT-Based Multi-Agent Paper Critique System

This module integrates all components of the PACT critique system:
- Input processing and paper parsing
- Supervisor planning and coordination
- Parallel evaluation by specialized dimension agents
- Synthesis of feedback into comprehensive critique
"""

from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage

from pact.state_pact_critique import PaperCritiqueState
from pact.pact_supervisor import plan_critique, synthesize_critique
from pact.pact_dimension_agents import (
    critique_research_foundations,
    critique_methodological_rigor,
    critique_structure_coherence,
    critique_academic_precision,
    critique_critical_sophistication
)

# ===== INPUT PROCESSING =====

def process_paper_input(state: MessagesState) -> dict:
    """
    Process the input paper from user messages.
    
    Extracts the paper content and any metadata provided.
    """
    # Get the last user message which should contain the paper
    messages = state.get('messages', [])
    if not messages:
        raise ValueError("No paper provided for critique")
    
    # Extract paper content from the last message
    last_message = messages[-1]
    if isinstance(last_message, HumanMessage):
        paper_content = last_message.content
    else:
        paper_content = str(last_message)
    
    # Extract title if provided in a specific format
    paper_title = None
    if paper_content.startswith("Title:"):
        lines = paper_content.split('\n')
        paper_title = lines[0].replace("Title:", "").strip()
    
    return {
        "paper_content": paper_content,
        "paper_title": paper_title
    }

# ===== PARALLEL DIMENSION EVALUATION =====

async def evaluate_dimensions(state: PaperCritiqueState) -> dict:
    """
    Coordinate parallel evaluation of all PACT dimensions.
    
    This node spawns all dimension agents to work in parallel.
    """
    # Run all dimension critiques in parallel
    import asyncio
    
    tasks = [
        critique_research_foundations(state),
        critique_methodological_rigor(state),
        critique_structure_coherence(state),
        critique_academic_precision(state),
        critique_critical_sophistication(state)
    ]
    
    # Wait for all tasks to complete
    results = await asyncio.gather(*tasks)
    
    # Combine all dimension critiques
    combined_critiques = {}
    for result in results:
        if 'dimension_critiques' in result:
            combined_critiques.update(result['dimension_critiques'])
    
    return {"dimension_critiques": combined_critiques}

# ===== GRAPH CONSTRUCTION =====

def build_pact_critique_graph():
    """
    Build the complete PACT critique workflow graph.
    """
    # Create the workflow
    workflow = StateGraph(PaperCritiqueState)
    
    # Add nodes
    workflow.add_node("process_input", process_paper_input)
    workflow.add_node("plan_critique", plan_critique)
    workflow.add_node("evaluate_dimensions", evaluate_dimensions)
    workflow.add_node("synthesize_critique", synthesize_critique)
    
    # Add edges
    workflow.add_edge(START, "process_input")
    workflow.add_edge("process_input", "plan_critique")
    # plan_critique uses Command to go to evaluate_dimensions
    workflow.add_edge("evaluate_dimensions", "synthesize_critique")
    workflow.add_edge("synthesize_critique", END)
    
    return workflow

# Compile the workflow
pact_critique_builder = build_pact_critique_graph()
pact_critique_agent = pact_critique_builder.compile()

## Test the PACT Critique System

In [ ]:
# Compile and visualize the workflow
from IPython.display import Image, display
from langgraph.checkpoint.memory import InMemorySaver
from pact.pact_critique_agent import pact_critique_builder

checkpointer = InMemorySaver()
pact_agent = pact_critique_builder.compile(checkpointer=checkpointer)
display(Image(pact_agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# Test with a sample paper excerpt
sample_paper = """
Title: The Impact of Social Media on Academic Performance: A Mixed Methods Study

Abstract:
This study examines the relationship between social media usage and academic performance among 
undergraduate students. Using surveys and interviews with 200 students, we found that excessive 
social media use correlates with lower GPA scores. However, educational use of social media 
platforms showed positive effects on collaborative learning.

Introduction:
Social media has become ubiquitous in student life. Many educators worry about its impact on 
academic performance. This study investigates whether these concerns are justified. Previous 
research has shown mixed results, with some studies finding negative correlations and others 
finding no significant relationship. Our research aims to clarify these conflicting findings.

Literature Review:
Smith (2020) found that students who spend more than 3 hours daily on social media have lower 
grades. Jones et al. (2019) argued that the type of social media use matters more than duration. 
Educational platforms like LinkedIn showed different patterns than entertainment-focused platforms 
like TikTok. However, these studies used different methodologies, making comparisons difficult.

Methodology:
We surveyed 200 undergraduate students about their social media habits and collected their GPA 
data. Additionally, we conducted 20 in-depth interviews to understand usage patterns. The survey 
included questions about daily usage time, platform preferences, and purposes of use.

Results:
Students using social media for more than 4 hours daily had an average GPA of 2.8, compared to 
3.2 for those using it less than 2 hours. However, students who used educational features had 
higher engagement scores in group projects.

Discussion:
Our findings suggest a nuanced relationship between social media and academic performance. While 
excessive recreational use appears detrimental, educational applications show promise. Universities 
should consider developing guidelines that encourage productive social media use.

Conclusion:
Social media's impact on academic performance depends on how it's used. Future research should 
explore interventions that promote beneficial usage patterns while minimizing distractions.
"""

# Run the critique
from langchain_core.messages import HumanMessage

thread = {"configurable": {"thread_id": "test_paper_1", "recursion_limit": 30}}
result = await pact_agent.ainvoke(
    {"messages": [HumanMessage(content=sample_paper)]}, 
    config=thread
)

In [ ]:
# Display the final critique
from rich.markdown import Markdown
if 'final_critique' in result:
    display(Markdown(result['final_critique']))
    print(f"\nOverall Score: {result.get('overall_score', 'N/A')}/100")
else:
    print("Critique not yet complete. Check state:", result.keys())

## Advanced Features: File Upload and Batch Processing

In [ ]:
%%writefile ../src/pact/pact_file_processor.py

"""
File Processing Utilities for PACT Critique System

Handles various file formats for paper submission.
"""

import os
from pathlib import Path
from typing import Optional
import PyPDF2
import docx

def read_paper_from_file(file_path: str) -> str:
    """
    Read paper content from various file formats.
    
    Supports: .txt, .pdf, .docx, .md
    """
    path = Path(file_path)
    
    if not path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")
    
    extension = path.suffix.lower()
    
    if extension == '.txt' or extension == '.md':
        with open(path, 'r', encoding='utf-8') as f:
            return f.read()
    
    elif extension == '.pdf':
        text = ""
        with open(path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text
    
    elif extension == '.docx':
        doc = docx.Document(path)
        return '\n'.join([paragraph.text for paragraph in doc.paragraphs])
    
    else:
        raise ValueError(f"Unsupported file format: {extension}")

def save_critique_to_file(critique: str, output_path: str, format: str = 'md') -> str:
    """
    Save critique to file in specified format.
    """
    path = Path(output_path)
    
    if format == 'md':
        path = path.with_suffix('.md')
        with open(path, 'w', encoding='utf-8') as f:
            f.write(critique)
    
    elif format == 'txt':
        path = path.with_suffix('.txt')
        # Convert markdown to plain text (basic conversion)
        plain_text = critique.replace('#', '').replace('*', '').replace('_', '')
        with open(path, 'w', encoding='utf-8') as f:
            f.write(plain_text)
    
    elif format == 'html':
        path = path.with_suffix('.html')
        import markdown
        html_content = markdown.markdown(critique)
        with open(path, 'w', encoding='utf-8') as f:
            f.write(f"""<!DOCTYPE html>
<html>
<head>
    <title>PACT Critique Report</title>
    <style>
        body {{ font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }}
        h1 {{ color: #333; }}
        h2 {{ color: #666; }}
        h3 {{ color: #888; }}
    </style>
</head>
<body>
{html_content}
</body>
</html>""")
    
    return str(path)

async def critique_paper_file(file_path: str, output_dir: Optional[str] = None) -> str:
    """
    Critique a paper from a file and save results.
    """
    from pact.pact_critique_agent import pact_critique_agent
    from langchain_core.messages import HumanMessage
    
    # Read paper content
    paper_content = read_paper_from_file(file_path)
    
    # Run critique
    result = await pact_critique_agent.ainvoke(
        {"messages": [HumanMessage(content=paper_content)]}
    )
    
    # Save critique if output directory specified
    if output_dir and 'final_critique' in result:
        output_path = Path(output_dir) / f"{Path(file_path).stem}_critique"
        saved_path = save_critique_to_file(
            result['final_critique'], 
            str(output_path), 
            format='md'
        )
        print(f"Critique saved to: {saved_path}")
    
    return result.get('final_critique', 'Critique generation failed')

## Usage Examples

In [ ]:
# Example: Process a paper from file
# Uncomment and modify path as needed
"""
from pact.pact_file_processor import critique_paper_file

# Critique a paper from a file
critique = await critique_paper_file(
    file_path="path/to/student_paper.pdf",
    output_dir="critique_reports/"
)

print("Critique generated successfully!")
"""

In [ ]:
# Create a simple CLI interface
%%writefile ../src/pact/pact_cli.py

"""
Command-line interface for PACT Critique System
"""

import asyncio
import argparse
from pathlib import Path

async def main():
    parser = argparse.ArgumentParser(description='PACT Academic Paper Critique System')
    parser.add_argument('input_file', help='Path to the paper file to critique')
    parser.add_argument('-o', '--output', help='Output directory for critique report')
    parser.add_argument('-f', '--format', choices=['md', 'txt', 'html'], 
                       default='md', help='Output format for critique')
    
    args = parser.parse_args()
    
    print(f"Processing paper: {args.input_file}")
    print("This may take a few minutes...")
    
    from pact.pact_file_processor import critique_paper_file
    
    try:
        critique = await critique_paper_file(
            file_path=args.input_file,
            output_dir=args.output
        )
        print("\n" + "="*50)
        print("Critique Complete!")
        print("="*50)
        
        if not args.output:
            print("\n" + critique)
    except Exception as e:
        print(f"Error: {e}")
        return 1
    
    return 0

if __name__ == "__main__":
    asyncio.run(main())

## Summary

This notebook has created a comprehensive PACT-based multi-agent critique system that:

1. **Loads and parses the PACT taxonomy** from the provided PACT_JSON.docx file
2. **Implements 5 specialized agents**, each focused on one PACT dimension
3. **Uses a supervisor agent** to coordinate the critique and synthesize feedback
4. **Provides structured, actionable feedback** with scores and recommendations
5. **Supports multiple file formats** for paper input (PDF, DOCX, TXT, MD)
6. **Generates formatted reports** in multiple output formats

The system can be:
- Used interactively in this notebook
- Deployed as a web service using LangGraph
- Run from the command line using the CLI interface
- Integrated into existing academic workflows

The modular design allows for easy customization and extension, such as:
- Adding more specialized agents for specific paper types
- Customizing rubrics for different academic levels
- Integrating with learning management systems
- Adding batch processing capabilities